In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import pandas as pd

import gridemissions as ge
from gridemissions.viz import set_plots
from gridemissions.viz.reports import myplot, heatmap

In [ ]:
ge.configure_logging("ERROR")
COLORS, PAGE_WIDTH, ROW_HEIGHT = set_plots()

In [ ]:
folder = ge.config["DATA_PATH"] / "EIA_Grid_Monitor" / "processed"
elec_files = [f for f in folder.iterdir() if f.name.endswith("elec.csv")]
raw_files = [f for f in folder.iterdir() if f.name.endswith("raw.csv")]

elec = ge.GraphData(
    pd.concat(
        [pd.read_csv(path, index_col=0, parse_dates=True) for path in elec_files],
        axis=0,
    )
)
raw = ge.GraphData(
    pd.concat(
        [pd.read_csv(path, index_col=0, parse_dates=True) for path in raw_files], axis=0
    )
)
raw.df.sort_index(inplace=True)
elec.df.sort_index(inplace=True)

# Note: we need to drop duplicate indices here because there seems to be duplicates at the beginning/end of the files
elec.df = elec.df[~elec.df.index.duplicated(keep="last")]
raw.df = raw.df[~raw.df.index.duplicated(keep="last")]

In [ ]:
deltas = elec.df - raw.df
delta_quantiles = deltas.quantile([0.1, 0.5, 0.9])
deltas_abs = deltas.abs().quantile([0.1, 0.5, 0.9])
median_raw = raw.df.abs().quantile(0.5)
delta_quantiles_pct = delta_quantiles / median_raw * 100

# Make plot

Select any timeseries below to see how it was adjusted

In [ ]:
def make_plot(col):
    s_abs = deltas[col]
    s_rel = s_abs / raw.df[col].abs() * 100

    f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(PAGE_WIDTH, ROW_HEIGHT * 3))
    ax1.set_title(col)
    # Relative plot (divide by demand)
    myplot(
        ax1,
        s_rel,
        COLORS[0],
        col,
    )
    ax1.set_ylabel("%")

    # Absolute plot (in MW)
    myplot(
        ax2,
        s_abs,
        COLORS[0],
        col,
    )
    ax2.set_ylabel("MW")

    # Heatmap (in MW)
    heatmap(
        s_abs,
        fax=(f, ax3),
        cmap="viridis",
        cbar_label="MW",
        transpose=True,
    )
    f.tight_layout()
    plt.show()


interact(make_plot, col=list(deltas.columns));